In [1]:
def get_medium():
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import glob
    import sklearn.preprocessing
    %matplotlib inline
    np.set_printoptions(threshold=np.nan)

    files = glob.glob("C:\\Users\\Zach\\Sync\\ML Class\\Project\\Data\\World Bank\\*.csv")
    i = 0
    start_year = '1971'
    end_year = '2014'
    series = []
    for file in files:
        if file[-14:-4] == 'population':
            df = pd.read_csv(file, index_col = 0)
            series.append(file[52:-4])
        else:
            df = pd.read_csv(file, index_col = 0, skiprows = 4)
            series.append(file[52:-4])
        df.head()
        if i == 0:
            data = df.loc['World',start_year:end_year].reset_index().values
            i = 1
        elif file[-14:-4] == 'population':
            df = df[df['Country Code'].str.match('WLD')]

            data = np.concatenate((data,df[df['Year'].between(int(start_year),int(end_year))].values[:,2].reshape(-1,1)),axis = 1)
        else:
            data = np.concatenate((data,df.loc['World',start_year:end_year].values.reshape(-1,1)),axis = 1)

    data = data[0:,1:]
    ecopd = pd.read_csv("C:\\Users\\Zach\\Sync\\ML Class\\Project\\Data\\ecological_footprint.csv", index_col = None)
    ecopd = ecopd.iloc[:,2:-2].drop(['Record'], axis = 1)
    data = np.concatenate((data[:,:], ecopd.iloc[10:,-1].values.reshape(-1,1)),axis = 1)
    series.append('ecological footprint')
    fossils = pd.read_csv("C:\\Users\\Zach\\Sync\\ML Class\\Project\\Data\\global-fossil-fuel-consumption.csv", index_col = 0)
    data = np.concatenate((data[:,:], fossils.loc[start_year:end_year,'Total'].values.reshape(-1,1)),axis = 1)
    series.append('fossil fuels')
    years = list(range(1971,1971+data.shape[0]))
    data = pd.DataFrame(data = data, columns = series, index = years)
    #data.to_csv(path_or_buf = "C:\\Users\\Zach\\Sync\\ML Class\\Project\\Data\\medium.csv" )
    return data

In [2]:
def get_big():
    df = pd.read_csv("C:\\Users\\Zach\\Sync\\ML Class\\Project\\Data\\world_data_1.csv")
    df = df.loc[:,:].replace('..', np.NaN)
    start_year = '1971'
    end_year = '2010'
    rows = np.where(np.prod(df.loc[:,start_year:end_year].notna().values, axis = 1))[0]
    df = df.iloc[rows,:]
    pred_steps = 14
    series = df.iloc[:,2].values
    df = df.loc[:,start_year:end_year]
    train = df.iloc[:,:].values.astype(float)
    years = df.loc[:,:].columns.values
    train = pd.DataFrame(data = train.T, columns = series, index = years)
    data = get_medium()
    years
    return data

In [3]:
import sklearn.preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import sklearn.preprocessing
%matplotlib inline
np.set_printoptions(threshold=np.nan)
data = get_medium()
data = data[['food production','gdp','population','ecological footprint','fossil fuels']]
scaler = sklearn.preprocessing.StandardScaler()
train = scaler.fit_transform(data)
train = train.reshape(1,-1,train.shape[1])
trainX = train[:,:30,:]
trainY = train[:,30,:]
#trainY = trainY.reshape(trainY.shape[1], trainY.shape[2])
testX = train[:,42-7:42,:]
testY = train[:,42:44,:]
#testY = testY.reshape(testY.shape[1], testY.shape[2])

# Sparse Identification

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from PDE_FIND import *
import scipy.io as sio
import itertools
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from scipy.integrate import solve_ivp

rhs_functions = {}
f = lambda x, y : np.prod(np.power(list(x), list(y)))
powers = []      
P = 2
d = data.shape[1]
for p in range(1,P+1):
        size = d + p - 1
        for indices in itertools.combinations(range(size), d-1):
            starts = [0] + [index+1 for index in indices]
            stops = indices + (size,)
            powers.append(tuple(map(operator.sub, stops, starts)))
for power in powers: rhs_functions[power] = [lambda x, y = power: f(x,y), power]
powers = list(rhs_functions.keys())
scaler = MinMaxScaler(feature_range=(1, 100))
# e = data['ecological footprint'].values  # vorticity
# p = data['population'].values/1000000000  # x-component of velocity
# ff = data['fossil fuels'].values/1000   # y-component of velocity

# ge = np.gradient(e)
# gp = np.gradient(p)
# gf = np.gradient(ff)

# e = scaler.fit_transform(e.reshape(-1,1)).flatten()
# p = scaler.fit_transform(p.reshape(-1,1)).flatten()
# ff = scaler.fit_transform(ff.reshape(-1,1)).flatten()

# ge = scaler.fit_transform(ge.reshape(-1,1)).flatten()
# gp = scaler.fit_transform(gp.reshape(-1,1)).flatten()
# gf = scaler.fit_transform(gf.reshape(-1,1)).flatten()

#Y = np.concatenate((ge.reshape(-1,1), gp.reshape(-1,1), gf.reshape(-1,1)), axis = 1)
#X_orig = np.concatenate((e.reshape(-1,1), p.reshape(-1,1), ff.reshape(-1,1)), axis = 1)
X_orig = scaler.fit_transform(data.values)
Y = np.gradient(X_orig, axis = 0)

#print(Y)
raised = []
#print(len(powers))
for power in powers:
    raised.append(np.power(X_orig,power))

X = raised
X = np.array(X)
X = np.prod(X,axis = 2)

clf = linear_model.Lasso(alpha=1, fit_intercept = True)
clf.fit(X.T,Y)
#print(powers)
coefs = clf.coef_
#print(coefs.T)
X = X.astype(float)

from scipy.integrate import odeint

def f(y,s, coefs, powers):
    raised = []
    ddt_out = []
    for power in powers:
        raised.append(np.power(s,power))
    X = raised
    X = np.array(X)
    X = np.prod(X,axis = 1)
    ddt = coefs @ X
    return [ddt]
s0 = X_orig[0,:]
t = np.linspace(1971,2040)
#s = odeint(f,s0,t, args = (coefs,powers))
sol = solve_ivp(lambda t, y: f(t, y, coefs,powers), [1971, 2040], s0, t_eval=t)
print(sol)
s = sklearn.preprocessing.scale(sol.y)
for i in range(s.shape[0]):
    plt.plot(t,s[i,:],'--', linewidth=2.0)
    plt.plot(range(1971,2015),sklearn.preprocessing.scale(X_orig[:,i]))
plt.xlabel("t")
plt.ylabel("S[N,C]")
#plt.legend(["Predicted Ecological Footprint","Predicted Population","Predicted Fossil Fuels","Actual Ecological Footprint",
#           "Actual Population", "Actual Fossil Fuels"])
plt.show()


In [ ]:
rhs_functions = {}
f = lambda x, y : np.prod(np.power(list(x), list(y)))
powers = []      
P = 3
d = 3
for p in range(1,P+1):
        size = d + p - 1
        for indices in itertools.combinations(range(size), d-1):
            starts = [0] + [index+1 for index in indices]
            stops = indices + (size,)
            powers.append(tuple(map(operator.sub, stops, starts)))
for power in powers: rhs_functions[power] = [lambda x, y = power: f(x,y), power]
coeffs = list(rhs_functions.keys())

In [ ]:
from sklearn import linear_model
import numpy as np
for i in range(data.shape[1]):
    Y = train[:,i]
    print(data.columns.values[i])
    dropped = data.drop(data.columns[i], axis = 1)
    X = np.delete(train, i, axis = 1)
    clf = linear_model.Lasso(alpha=0.1, fit_intercept = False)
    clf.fit(X,Y)
    
    
    print(dropped.columns.values[np.nonzero(clf.coef_)[0]])
    print(clf.coef_[np.nonzero(clf.coef_)[0]])

In [ ]:
import sklearn.preprocessing

train = sklearn.preprocessing.scale(small_data)
for i in range(train.shape[1]-1):
    plt.plot(range(1971,train.shape[0]+1971),train[:,i].astype(np.float64))

In [ ]:
import numpy as np
train = data.values.astype(np.float64)
U, s, Vh = np.linalg.svd(data.values.astype(np.float64), full_matrices=False)
s[2:] = 0
new_a = np.dot(U, np.dot(np.diag(s), Vh))
for i in range(train.shape[1]):
    plt.plot(range(1971,train.shape[0]+1971), train[:,i])
    plt.plot(range(1971,train.shape[0]+1971), new_a[:,i])
    plt.show()

# Dynamic Mode Decomposition

In [ ]:
from pydmd import DMD
import scipy.integrate
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib

SMALL_SIZE = 15
matplotlib.rc('font', size=SMALL_SIZE)
matplotlib.rc('axes', titlesize=SMALL_SIZE)
dmd = DMD(svd_rank=2)
dmd.fit(train[:-7,:].T)
output = dmd.reconstructed_data
A = dmd.atilde
#plt.plot(range(0,44), train[:,3])
#plt.plot(range(0,44), np.real(output.T[:,3]))
#plt.show()
#mae = mean_squared_error(sklearn.preprocessing.scale(train[:,3]), sklearn.preprocessing.scale(np.real(output.T[:,3])))
dmd.dmd_time['tend'] += 7
output = dmd.reconstructed_data
error_list = []
i = 3
plt.figure(figsize=(10,6))
outplot = np.real(output.T[:,i])
plt.plot(range(1971,train.shape[0]+1971), train[:,i])
plt.plot(range(1971,train.shape[0]+1971-7), outplot[:-7])
plt.plot(range(train.shape[0]+1971-7,train.shape[0]+1971), outplot[-7:], linestyle='--')
plt.title('Dynamic Mode Decomposition Prediction of ' + data.columns.values[i])
plt.xlabel('Year')
plt.ylabel('Ecological Footprint (Earths)')
#plt.savefig(data.columns.values[i] + ' dmd.pdf', bbox_inches='tight')

plt.show()
mae = (mean_squared_error(train[-7:,i], np.real(output.T)[-7:,i]))
error_list.append(mae)
print(error_list)

In [ ]:
# VAR example
import statsmodels.api as sm
# contrived dataset with dependency

print(data['ecological footprint'].values)
# Fit the model
mod = sm.tsa.statespace.SARIMAX(data['ecological footprint'].values, trend='c', order=(1,1,1))
res = mod.fit(disp=False)
print(res.summary())

# fit model
model = VAR(train[:-14,:])
model_fit = model.fit()
# make prediction
yhat = model_fit.forecast(model_fit.y, steps=14)
#print(yhat)
print(model_fit.score(model_fit.y, steps = 14))
for i in range(train.shape[1]):
    plt.plot(range(1,15), train[-14:,i])
    plt.plot(range(1,15), yhat[:,i])
    plt.legend(['actual','pred'])
    plt.show()


In [ ]:
"""
@author: Maziar Raissi
"""


import numpy as np
from scipy.integrate import odeint

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#from plotting import newfig, savefig
import matplotlib.gridspec as gridspec

if __name__ == "__main__": 
    
    # function that returns dx/dt
    def f(x,t): # x is 3 x 1
        J0 = 2.5
        k1 = 100.0
        k2 = 6.0
        k3 = 16.0
        k4 = 100.0
        k5 = 1.28
        k6 = 12.0
        k = 1.8
        kappa = 13.0
        q = 4
        K1 = 0.52
        psi = 0.1
        N = 1.0
        A = 4.0
        
        f1 = J0 - (k1*x[0]*x[5])/(1 + (x[5]/K1)**q)
        f2 = 2*(k1*x[0]*x[5])/(1 + (x[5]/K1)**q) - k2*x[1]*(N-x[4]) - k6*x[1]*x[4]
        f3 = k2*x[1]*(N-x[4]) - k3*x[2]*(A-x[5])
        f4 = k3*x[2]*(A-x[5]) - k4*x[3]*x[4] - kappa*(x[3]-x[6])
        f5 = k2*x[1]*(N-x[4]) - k4*x[3]*x[4] - k6*x[1]*x[4]
        f6 = -2*(k1*x[0]*x[5])/(1 + (x[5]/K1)**q) + 2*k3*x[2]*(A-x[5]) - k5*x[5]
        f7 = psi*kappa*(x[3]-x[6]) - k*x[6]
        
        f = np.array([f1,f2,f3,f4,f5,f6,f7])
        return f
        
    # time points
    t_star = np.arange(1971,2015,1)
    
    S1 = np.random.uniform(0.15,1.60,1)
    S2 = np.random.uniform(0.19,2.16,1)
    S3 = np.random.uniform(0.04,0.20,1)
    S4 = np.random.uniform(0.10,0.35,1)
    S5 = np.random.uniform(0.08,0.30,1)
    S6 = np.random.uniform(0.14,2.67,1)
    S7 = np.random.uniform(0.05,0.10,1)
    
    # initial condition
    X_train = send_train()
    x0 = np.array(X_train[0,:]).flatten()
    
    # solve ODE
    #X_star = odeint(f, x0, t_star)
    
    noise = 0.00
    
    skip = 1
    dt = 1
    #X_train = X_star[0::skip,:]
    X_train = send_train()
    #X_train = X_train + noise*X_train.std(0)*np.random.randn(X_train.shape[0], X_train.shape[1])
    
    X_train = np.reshape(X_train, (1,X_train.shape[0],X_train.shape[1]))
    print(X_train.shape)
    layers = [19, 256, 19]
    
    M = 1
    scheme = 'AM'
    model = Multistep_NN(dt, X_train, layers, M, scheme)
    
    N_Iter = 50000
    model.train(N_Iter)
    
    def learned_f(x,t):
        f = model.predict_f(x[None,:])
        return f.flatten()
    print(x0.shape)
    print(t_star.shape)
    print(learned_f)
    learned_X_star = odeint(learned_f, x0.astype(float), t_star.astype(float))
    print(learned_X_star[:,-1])
    ####### Plotting ##################
    
    plt.plot(t_star, learned_X_star[:,:])
    
    fig, ax = newfig(1.0, 1.55)
    ax.axis('off')
    
    gs0 = gridspec.GridSpec(3, 2)
    gs0.update(top=0.95, bottom=0.35, left=0.1, right=0.95, hspace=0.5, wspace=0.3)
    
    ax = plt.subplot(gs0[0:1, 0:1])
    ax.plot(t_star,X_star[:,0],'r-')
    ax.plot(t_star,learned_X_star[:,0],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$S_1$')
    
    ax = plt.subplot(gs0[0:1, 1:2])
    ax.plot(t_star,X_star[:,1],'r-')
    ax.plot(t_star,learned_X_star[:,1],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$S_2$')

    ax = plt.subplot(gs0[1:2, 0:1])
    ax.plot(t_star,X_star[:,2],'r-')
    ax.plot(t_star,learned_X_star[:,2],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$S_3$')
    
    ax = plt.subplot(gs0[1:2, 1:2])
    ax.plot(t_star,X_star[:,3],'r-')
    ax.plot(t_star,learned_X_star[:,3],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$S_4$')
    
    ax = plt.subplot(gs0[2:3, 0:1])
    ax.plot(t_star,X_star[:,4],'r-')
    ax.plot(t_star,learned_X_star[:,4],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$S_5$')
    
    ax = plt.subplot(gs0[2:3, 1:2])
    ax.plot(t_star,X_star[:,5],'r-')
    ax.plot(t_star,learned_X_star[:,5],'k--')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$S_6$')

    gs1 = gridspec.GridSpec(1, 2)
    gs1.update(top=0.25, bottom=0.105, left=0.325, right=0.7, hspace=0.5, wspace=0.3)
    
    ax = plt.subplot(gs1[0:1, 0:2])
    ax.plot(t_star,X_star[:,6],'r-',label='Exact Dynamics')
    ax.plot(t_star,learned_X_star[:,6],'k--',label='Learned Dynamics')
    ax.set_xlabel('$t$')
    ax.set_ylabel('$S_7$')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=2, frameon=False)

# savefig('./figures/Glycolytic', crop = False)

In [ ]:
X_train = send_train()
#X_train = X_train + noise*X_train.std(0)*np.random.randn(X_train.shape[0], X_train.shape[1])
    
X_train = np.reshape(X_train, (1,X_train.shape[0],X_train.shape[1]))
print(X_train)

In [ ]:
t_star = np.arange(1970,2016,1)
print(t_star)